In [18]:
import taxon
import gui_widgets
from wikidataintegrator import wdi_core
import bibtexparser
import requests
import pandas as pd

In [19]:
import ipywidgets as widgets
from IPython.display import IFrame
tab1 = widgets.Output()
tab2 = widgets.Output()
tab3 = widgets.Output()
tab4 = widgets.Output()
tab5 = widgets.Output()
tab6 = widgets.Output()
tab = widgets.Tab(children=[tab1,tab2, tab3, tab4, tab5, tab6])
tab

iNaturalistTab = IFrame(src='https://www.inaturalist.org/home', width=1000, height=600)
tab.set_title(0, 'iNaturalist') 
tab.set_title(1, 'GBIF')
tab.set_title(2, 'Wikidata')
tab.set_title(3, 'BHL')
tab.set_title(4, 'Commons')
tab.set_title(5, 'Wikipedia')

with tab1:
    def paste_commons(commons_file_name):
        with tab6:
                print("https://en.wikipedia.org/wiki/"+data.inaturalist_data[0]["name"].replace(" ", "_"))
                print("=========================")
                print(data.create_wikipedia_stub(infobox_image=commons_file_name)) 
        return commons_file_name
    def get_data(taxon_id):
        global data
        data = taxon.external_data(inaturalist_id=taxon_id)
        html = "<img src='"+data.inaturalist_data[0]['default_photo']['medium_url']+"'><br>"+data.inaturalist_data[0]['default_photo']['attribution']
        output_widget = widgets.HTML(value=html)
        with tab2:
            html2 = "<table>"
            for key in data.gbif_data.keys():
                html2 += "<tr><td>{}</td><td>{}</td></tr>".format(key, data.gbif_data[key])
            html2 += "</table>"
            gbif_output = widgets.HTML(value=html2)
            display(gbif_output)
        with tab4:
            bhlurl = "https://www.biodiversitylibrary.org/name/"+data.inaturalist_data[0]["name"].replace(" ", "_")
            print("source: ", bhlurl)
            bhlbibtexurl = "https://www.biodiversitylibrary.org/namelistdownload/?type=b&name="+data.inaturalist_data[0]["name"].replace(" ", "_")
            bibtex = requests.get(bhlbibtexurl).text
            bib_database = bibtexparser.loads(bibtex)
            fields = []
            for entry in bib_database.entries:
                for key in entry.keys():
                    if key not in fields:
                        fields.append(key)
            fields
            df = pd.DataFrame(columns= fields)
            df.style.set_caption("Hello World")
            for i in range(len(bib_database.entries)):
                row = dict()
                for key in fields:
                    if key not in bib_database.entries[i].keys():
                        row[key]=None
                    else:
                        row[key]=bib_database.entries[i][key]
                df.loc[i] = row
            display(df)
        with tab5:
            commons_query = """
                SELECT * WHERE {{?commons schema:about <{taxon}>  ; 
                schema:isPartOf <https://commons.wikimedia.org/> . 
                }}""".format(taxon = data.wikidata["main_rank"].loc[0]["taxon"])
            commons_query_result = wdi_core.WDItemEngine.execute_sparql_query(commons_query, as_dataframe=True)
            if len(commons_query_result) == 0:
                html5 = "<a href = 'https://commons.wikimedia.org/w/index.php?title=Category:"+data.inaturalist_data[0]["name"].replace(" ", "_")+"&action=edit'>create commons category</a><br>"
                html5 += "[[Category:"+data.inaturalist_data[0]["name"].replace(" ", "|")+"]]"
            else:
                html5 = "<a href = 'https://commons.wikimedia.org/wiki/Category:"+data.inaturalist_data[0]["name"].replace(" ", "_")+"' target='_new'>"+data.inaturalist_data[0]["name"].replace(" ", "_")+"</a><br>"
            commons_output = widgets.HTML(value=html5)
            
            data.selected_commons=gui_widgets.interact_manual(paste_commons, commons_file_name="")
            display(commons_output)
            
        return output_widget

    
    data = None
    taxon_window = gui_widgets.interact_manual(get_data, taxon_id="")
    
    
    



display(tab)